<a href="https://colab.research.google.com/github/HienNguyenMinh/PythonAssignment/blob/main/BaiTap1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Học viên: Nguyễn Thị Minh Hiền

Khóa: MSE11_HCM


#Giải Thích Để Hiểu Về M5-Forecasting Bằng Tiếng Việt
#Giới thiệu:
Trong cuộc thi tên là M5-Forecasting, những người tham gia cuộc thi sử dụng dữ liệu bán hàng phân cấp được trích xuất từ Walmart cho việc phân tích và dự đoán doanh số hàng ngày trong 28 ngày tới. *(Walmart là công ty lớn nhất thế giới theo doanh thu)*.
Tập dữ liệu bán hàng chứa thông tin của khoảng 30.000 mặt hàng khác nhau trong gần 1900 ngày. Dữ liệu được phân bổ quanh các cửa hàng ở ba tiêu bang Hoa Kỳ (California, Texa và Wisconsin) bao gồm các cấp mặt hàng, bộ phận, danh mục sản phẩm và cấp chi tiết cửa hàng. Ngoài ra, tập dữ liệu cũng có các biến số giải thích về giá cả, các chương trình khuyến mãi, ngày trong tuần và các sự kiện đặc biệt. Hơn nữa, tập dữ liệu mạnh mẽ này không chỉ giúp cải thiện độ chính xác của dự báo, mà còn giúp hiển thị các mô hình bán hàng cơ bản thông qua các ngôn ngữ khác nhau cũng như dựa trên các danh mục sản phẩm khác nhau cung cấp giá trị cho các tầm nhìn kinh doanh.

Theo thực tế là một tập dữ liệu lớn được tạo sẵn, chúng ta bắt đầu với một công thức kinh doanh được đưa ra xoay quanh vài vấn đề. Liệu tập dữ liệu này có đủ để trả lời các câu hỏi đó hay không. Nếu câu trả lời chưa được tìm thấy từ tập dữ liệu này, thì ít nhất các thông tin cần thiết theo tập điểm dữ liệu cũng giúp cải thiện việc thiết kế dữ liệu trong tương lai. 

#Báo cáo vấn đề
Một tập dữ liệu lớn khiến cho việc tìm kiếm tất cả các mẫu cơ bản trong tập dữ liệu đó trở nên khó khăn. May mắn thay, việc đặt ra các hỏi có tính chất SMART rất hữu ích với việc hiểu các mẫu mà có thể vô tình không nhìn thấy nó trong lần đầu tiếp cận vào tập dữ liệu. *(SMART: cụ thể, có thể đo lường, có thể đạt được, có liên quan và có giới hạn thời gian)*.

Trong các vấn đề cụ thể của m5-forecasting đặc biệt quan tâm việc tìm ra câu trả lời cho các câu hỏi sau:

* Sự phân phối chung của các ID mặt hàng trên các danh mục là gì?
* Hành vi của các danh mục trên các cửa hàng khác nhau là gì?
* ID của mặt hàng bán chạy nhất?
* ID của mặt hàng trong mỗi danh mục có nhiều doanh thu nhất?
* Doanh thu bán hàng thu được nhiều nhất là bao nhiêu? Nó có phải là ID của các mặt hàng bán chạy nhất, mang lại doanh thu cao nhất, hoặc có thay đổi theo xu hướng không?
* Hành vi của tổng doanh số bán hàng là gì?
* Hành vi của ID các mặt hàng đang bán trong các ngày khác nhau trong tuần là gì? Có ngày cụ thể nào mà hôm đó có doanh thu cao nhất? Xu hướng thay đổi như thế nào ở các cửa hàng khác nhau?

Và cuối cùng, chúng tôi sẽ sử dụng SARIMAX và Facebook đã phát triển mô hình Prophet để dự báo doanh số bán hàng trong 28 ngày tới.

#Các chỉ số đánh giá:
Để đưa ra dự báo một chuổi thời gian (time series), chung ta dùng các error metrics sau:
1. RMSE (Root Mean Square Error): Để đánh giá dự báo bán hàng được thực hiện bằng SARIMAX, RMSE error metric được sử dụng. Chuỗi thời gian được sử dụng trong trường hợp này, lấy số lượng bán hàng trung bình được thực hiện, trong suốt gần 1900 ngày, bỏ qua việc sắp xếp dữ liệu thứ cấp ban đầu của id sản phẩm cùng với một số cấp độ khác. Mức độ tối đa của error metric RMSE không bị vô hiệu vì chúng tôi đang xem xét số lượng bán hàng trung bình được thực hiện, do đó tận dụng hiệu quả của rất nhiều số 0 đối với nhiều id sản phẩm, bên cạnh số ngày.
2. Custom Loss (WMAPE): error metric này được sử dụng cho những trường hợp mà mức độ ưu tiên của ID sản phẩm được xem xét cùng với doanh số đã thực hiện. Các trọng số được tính toán bằng cách chia sự khác biệt giữa dự báo và giá trị bán hàng thực tế cho giá trị trung bình của chúng.
3. WRMSSE (Weighted Root Mean Square Scaled Error): WRMSSE được tính khi khung dữ liệu sẵn sàng dự đoán (dự báo gần 30.000 id sản phẩm) đã sẵn sàng. Việc tính toán WRMSSE yêu cầu phải sử dụng tổng số mục nhập của tất cả id sản phẩm (gần 30.000 sản phẩm) và sau đó là sự khác biệt giữa các giá trị của tập dữ liệu xác thực, tức là sales_train_validation.csv và khung dữ liệu được dự báo. WRMSSE eror metirc này là chỉ số do cuộc thi cung cấp và đã được tối ưu hóa để sử dụng cho cuộc thi hiện tại. Để đánh giá nó, chúng tôi sẽ phải sử dụng toàn bộ tập dữ liệu của sales_train_validation.csv đã được cung cấp trong một tệp csv.


Tải các thư viện cần dùng: 

* Tải các module như numpy, pandas, pyplot, time, math, datetime, widgets..Dùng dòng lệnh: <font color='blue'> import <tên module> </font> hoặc <font color='blue'>import <tên module> as <định danh></font>

* Tải các hàm như display, KDTree...Dùng dòng lệnh: <font color='blue'>from <tên module> import <tên hàm></font>

In [22]:
# Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import time
import math
import datetime


# Import widgets
from ipywidgets import widgets, interactive, interact
import ipywidgets as widgets
from IPython.display import display

from math import log, floor
from sklearn.neighbors import KDTree

import seaborn as sns
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.stattools import adfuller

import itertools
from itertools import cycle
plt.style.use('seaborn')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])


Tải các file dữ liệu dùng cho việc phân tích và dự báo về google Colab bằng cách clone các file này từ một reponsitory trên GitHub:

In [25]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/HienNguyenMinh/PythonAssignment.git


Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 1.6.14
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Need to get 7,168 kB of archives.
After this operation, 7,962 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ub

In [24]:
!rm -rf /content/PythonAssignment

Kiểm tra các file dữ liệu *.csv được tải về Colab thông qua việc duyệt cây thư mục vừa được tải về Colab và in ra đường dẫn các file dữ liệu: 

In [12]:
import os
for dirname, _, filenames in os.walk('/content/PythonAssignment/DataSet'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/PythonAssignment/DataSet/sales_train_evaluation.csv
/content/PythonAssignment/DataSet/calendar.csv
/content/PythonAssignment/DataSet/sell_prices.csv
/content/PythonAssignment/DataSet/sales_train_validation.csv
/content/PythonAssignment/DataSet/sample_submission.csv
/content/PythonAssignment/DataSet/.gitattributes


Mô tả tập dữ liệu:

Các file data được tải vào notebook này như sau:

* calendar_df.csv - chứa các thông tin về những ngày mà sản phẩm được bán, thông tin về những ngày lễ và các dịp đặc biệt.
* train_sales_df.csv - Chứa dữ liệu lịch sử bán hàng hàng ngày cho mỗi sản phẩm và cửa hàng và ID bộ phận với gần như dữ liệu bán hàng trong 1900 ngày [d_1 - d_1913]
* submission_file.csv - Định dạng đúng cho tệp gửi, chứa id sản phẩm và id cột cho dự báo dữ liệu bán hàng 28 ngày tới.
* sell_prices_df.csv - chứa thông tin về giá của mỗi sản phẩm đã bán ở mỗi cửa hàng và ngày.
* sales_train_evaluation.csv - bao gồm các nhãn hàng [d_1 - d_1941] 

Đọc dữ liệu từ các file *.csv bằng hàm read_csv() của pandas:

In [19]:
sell_prices_df = pd.read_csv('/content/PythonAssignment/DataSet/sell_prices.csv')
train_sales_df = pd.read_csv('/content/PythonAssignment/DataSet/sales_train_validation.csv')
calendar_df = pd.read_csv('/content/PythonAssignment/DataSet/calendar.csv')
submission_file = pd.read_csv('/content/PythonAssignment/DataSet/sample_submission.csv')

In full summary của dataframe để hiểu tập dữ liệu:

In [20]:
sell_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   version https://git-lfs.github.com/spec/v1  2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


In [16]:
train_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   version https://git-lfs.github.com/spec/v1  2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


In [17]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB
